In [ ]:
!pip install yfinance
import yfinance as yf
import pandas as pd

In [ ]:
url = "https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average"
print(url)

df = pd.read_html(url)
print(df)
print(df.info())

In [ ]:
df.rename(columns = {"Date added":"Date_Added"}, inplace = True)
df.rename(columns = {"Index weighting":"Weights"}, inplace = True)

In [ ]:
df.Date_Added = pd.to_datetime(df.Date_Added)
print(df)

df.Weights = pd.to_numeric(df.weights.str.replace("%", ""))
print(df)

df.Weights.sum()
print(df)

In [ ]:
df.drop(columns = "Notes", inplace = True)
print(df)

df.set_index("Symbol", inplace = True)
print(df)

symbols = df.index.to_list()
print(symbols)

last_update = df["Date_Added"].max()
print(last_update)

In [ ]:
ts = yf.download(tickers = symbols, start = last_update)
print(ts)
print(ts.info())
print(ts.Close)
print(ts.dropna(inplace = True))

perf = ts.Close.iloc[-1].div(ts.Close.iloc[0]).sub(1).sort_values(ascending = False)
print(perf) # total price increase/decrease (in %) since last reconstitution

perf.index.name = "Symbol"
print(perf)



More Detialed Stock Data

In [ ]:
msft = yf.Ticker(ticker = "msft")
print(msft)
print(msft.get_info())

info = pd.Series(msft.get_info()).to_frame().T
print(info)

cs = pd.DataFrame()
print(cs)

count = 1
for symbol in symbols:
  try:
    info = pd.Series(yf.Ticker(ticker = symbol).get_info()).to_frame().T
    cs = pd.concat([cs,info])
    print(count, end = '\r')
    ccount+=1
  except Exception as e:
    print("{} not found").format(symbol)
  print("Download complete.")



Equity Analysis & Comparison

In [ ]:
from pandas.io.formats.info import DataFrameTableBuilderVerbose
print(cs["priceToBook"].sort_values(ascending = False))
print(cs.loc["BA", "bookValue"])

df["PricetoBook"] = (cs.currentPrice / cs.bookValue)
print(df)

print(cs.dividendYield)

df["Dividend_Yield"] = cs.dividendYield.fillna(0)
print(DataFrameTableBuilderVerbose)

df["ForwardPE"] = cs.forwardPE
print(df)

df.sort_values("PricetoBook", ascending = False)
df.sort_values("ForwardPE", ascending = False)
df.sort_values("Dividend_Yield", ascending = False)
